In [1]:
from experiments.utilities import *

In [2]:
data_repository = '/Users/maoss2/PycharmProjects/BRCA_experiments_and_paper/datasets/datasets_repository'
dico = defaultdict(dict)
ensemble_name_to_gene_name_file = f"{data_repository}/9606.protein.info.v11.0.txt"
string_db_prot_link_file = f"{data_repository}/9606.protein.links.v11.0.txt"

In [3]:
string_db_link = pd.read_csv(string_db_prot_link_file, sep=' ')
string_db_link = string_db_link.loc[string_db_link['combined_score'] >= 500] # threshold score
ensemble_name_to_gene_name = pd.read_csv(ensemble_name_to_gene_name_file, sep='\t')


In [4]:
string_db_link.head()

,protein1,protein2,combined_score
3,9606.ENSP00000000233,9606.ENSP00000418915,606
15,9606.ENSP00000000233,9606.ENSP00000432568,909
16,9606.ENSP00000000233,9606.ENSP00000427900,910
24,9606.ENSP00000000233,9606.ENSP00000350199,751
35,9606.ENSP00000000233,9606.ENSP00000438085,614


In [5]:
ensemble_name_to_gene_name.head()

,protein_external_id,preferred_name,protein_size,annotation
0,9606.ENSP00000000233,ARF5,180,ADP-ribosylation factor 5; GTP-binding protein...
1,9606.ENSP00000000412,M6PR,277,Cation-dependent mannose-6-phosphate receptor;...
2,9606.ENSP00000001008,FKBP4,459,Peptidyl-prolyl cis-trans isomerase FKBP4; Imm...
3,9606.ENSP00000001146,CYP26B1,512,Cytochrome P450 26B1; Involved in the metaboli...
4,9606.ENSP00000002125,NDUFAF7,441,"Protein arginine methyltransferase NDUFAF7, mi..."


In [6]:
temp_dict = {cle: ensemble_name_to_gene_name['preferred_name'].values[idx] for idx, cle in enumerate(ensemble_name_to_gene_name['protein_external_id'].values)}

In [7]:
ensbl_to_ensbl_link_dict = {}
for idx, cle in enumerate(string_db_link['protein1'].values):
    if cle in ensbl_to_ensbl_link_dict.keys():
        ensbl_to_ensbl_link_dict[cle].append(string_db_link['protein2'].values[idx])
    else:
        ensbl_to_ensbl_link_dict[cle] = [string_db_link['protein2'].values[idx]]

In [8]:
print(len(list(ensbl_to_ensbl_link_dict.keys())))
print(len(list(temp_dict.keys())))
# for k, values in ensbl_to_ensbl_link_dict.items():
new_dict = {}
list_cles = list(temp_dict.keys())
list_values = list(temp_dict.values())

19146
19566


In [9]:
for cle, values in ensbl_to_ensbl_link_dict.items():
    if cle in list_cles:
        new_values = [temp_dict[new_cle] for new_cle in values]
        new_values = [el for el in new_values if not el.startswith('ENSG0')]
        new_dict[temp_dict[cle]] = new_values

In [10]:
string_pathways = [list(v) + [k] for k, v in new_dict.items()]
string_pathways.append(['miRNA']) #'G_19139'
string_pathways.append(['clinical View']) #'G_19140'
string_pathways.append(['unknown']) # 'G_19141'

In [11]:
with open(f'{data_repository}/string_pathways_list.pck', 'wb') as f:
    pickle.dump(string_pathways, f)

In [12]:
data_tn_new_label_unbalanced_cpg_rna_rna_iso_mirna = f"{data_repository}/triple_neg_new_labels_unbalanced_cpg_rna_rna_iso_mirna.h5"
_, _, features_names, _ = load_data(data=data_tn_new_label_unbalanced_cpg_rna_rna_iso_mirna, 
                                    return_views='all')

In [13]:
features_names = list(features_names)
dico_results = {feature: [] for feature in features_names}

In [14]:
for feature in features_names:
    if feature.find('_') != -1:
        # I went step by step for the comprehension but remember the gene is always at the end of the feature so
        # use the [-1] access
        split_results = feature.split('_')
        gene_cible = split_results[-1]
        if gene_cible.find(';'): # Cas ou on a un feature lié à 2 ou plus genes (surtout pour les cpg)
            gene_cibles = gene_cible.split(';')
            for gene in gene_cibles: # Recupere chaque gene et on remplit le dico
                for idx, pathway_in_biogrid in enumerate(string_pathways):
                    if gene in pathway_in_biogrid:
                        dico_results[feature].append('G_{}'.format(idx))
        else: # Different du premier if du coup le feature est link à un seul gene et on remplit le dictionnaire de facon adéquate
            for idx, pathway_in_biogrid in enumerate(string_pathways):
                if gene_cible in pathway_in_biogrid:
                    dico_results[feature].append('G_{}'.format(idx))
    elif feature.find('|') != -1: # Here the gene is the 1st element always since it's directly the RNA view only
        split_results = feature.split('|')
        gene_cible = split_results[0]
        for idx, pathway_in_biogrid in enumerate(string_pathways):
            if gene_cible in pathway_in_biogrid:
                dico_results[feature].append('G_{}'.format(idx))
    elif feature.startswith('hsa'):  # MiRNA View: faire le traitement directement
        print(feature)
        dico_results[feature].append('G_{}'.format(len(string_pathways) - 3))
        print(dico_results[feature])
    else:
        dico_results[feature].append('G_{}'.format(len(string_pathways) - 2))
for cle, valeur in dico_results.items():
    if valeur == []:
        dico_results[cle].append('G_{}'.format(len(string_pathways) - 1))


hsa-mir-1307
['G_19139']
hsa-mir-542
['G_19139']
hsa-mir-139
['G_19139']
hsa-mir-432
['G_19139']
hsa-mir-410
['G_19139']
hsa-mir-22
['G_19139']
hsa-mir-615
['G_19139']
hsa-mir-382
['G_19139']
hsa-mir-1180
['G_19139']
hsa-mir-140
['G_19139']
hsa-let-7a-3
['G_19139']
hsa-mir-154
['G_19139']
hsa-mir-30d
['G_19139']
hsa-mir-181a-2
['G_19139']
hsa-mir-103-1
['G_19139']
hsa-mir-135b
['G_19139']
hsa-mir-744
['G_19139']
hsa-mir-425
['G_19139']
hsa-let-7f-1
['G_19139']
hsa-mir-224
['G_19139']
hsa-mir-210
['G_19139']
hsa-mir-16-1
['G_19139']
hsa-mir-30a
['G_19139']
hsa-mir-9-2
['G_19139']
hsa-mir-324
['G_19139']
hsa-mir-21
['G_19139']
hsa-mir-184
['G_19139']
hsa-mir-193a
['G_19139']
hsa-mir-197
['G_19139']
hsa-let-7a-1
['G_19139']
hsa-mir-589
['G_19139']
hsa-mir-26a-1
['G_19139']
hsa-mir-203
['G_19139']
hsa-mir-409
['G_19139']
hsa-mir-152
['G_19139']
hsa-mir-181c
['G_19139']
hsa-mir-101-1
['G_19139']
hsa-mir-660
['G_19139']
hsa-mir-412
['G_19139']
hsa-mir-18a
['G_19139']
hsa-mir-338
['G_19139']


In [15]:
dico_results = {k: list(np.unique(v)) for k, v in dico_results.items()}
with open(f'{data_repository}/pathways_string_groups.pck', 'wb') as f:
    pickle.dump(dico_results, f)

In [16]:
temp_dict = {f'G_{idx}': el for idx, el in enumerate(string_pathways)}

In [17]:
with open(f'{data_repository}/string_pathways_dict.pck', 'wb') as f:
    pickle.dump(temp_dict, f)

In [18]:
dico_results['hsa-mir-1307']

['G_19139']